In [1]:
# create bucket - lpkadvworks
!gsutil mb gs://lpkadvworks

Creating gs://lpkadvworks/...


In [2]:
!gsutil ls 

gs://lpkadvworks/
gs://lpkretail/


In [4]:
# copy data directory to GCS bucket
!gsutil cp -r ../data gs://lpkadvworks

Copying file://..\data\AdventureWorks_Calendar.csv [Content-Type=application/vnd.ms-excel]...
/ [0 files][    0.0 B/  9.7 KiB]                                                
/ [1 files][  9.7 KiB/  9.7 KiB]                                                
Copying file://..\data\AdventureWorks_Customers.csv [Content-Type=application/vnd.ms-excel]...
/ [1 files][  9.7 KiB/  1.9 MiB]                                                
-
- [2 files][  1.9 MiB/  1.9 MiB]                                                
Copying file://..\data\AdventureWorks_Products.csv [Content-Type=application/vnd.ms-excel]...
- [2 files][  1.9 MiB/  1.9 MiB]                                                
- [3 files][  1.9 MiB/  1.9 MiB]                                                
\
Copying file://..\data\AdventureWorks_Product_Categories.csv [Content-Type=application/vnd.ms-excel]...
\ [3 files][  1.9 MiB/  1.9 MiB]                                                
\ [4 files][  1.9 MiB/  1.9 MiB]          

In [6]:
# validate for files in bucket
!gsutil ls -r gs://lpkadvworks

gs://lpkadvworks/data/:
gs://lpkadvworks/data/AdventureWorks_Calendar.csv
gs://lpkadvworks/data/AdventureWorks_Customers.csv
gs://lpkadvworks/data/AdventureWorks_Product_Categories.csv
gs://lpkadvworks/data/AdventureWorks_Product_Subcategories.csv
gs://lpkadvworks/data/AdventureWorks_Products.csv
gs://lpkadvworks/data/AdventureWorks_Returns.csv
gs://lpkadvworks/data/AdventureWorks_Territories.csv

gs://lpkadvworks/data/AW_Sales/:
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2015.csv
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2016.csv
gs://lpkadvworks/data/AW_Sales/AdventureWorks_Sales_2017.csv
